# Keyword Tagging & Normalization Skeleton
## [1.Data Extraction_&_Preprocessing](#1.Data_Extraction_&_Preprocessing)

### Dataset
### Type of Dataset: Patent Data Abstract: Patbase
### Size of Dataset: 2000k Records

### Preprocessing

1) We used <b>NLTK</b> to tokenize and Lemmatize the Text 

### Improvements

#### Word Tokenization and Lemmatization can be improved with POS Tagging

## [2.Keyword_&_Phrase_Extraction](#2.Keyword_&_Phrase_Extraction)

1) Used <b>Count Vectorizer and Tfidf</b> to get candidate keywords \
2) <b>Paraphrase mining sentence embedding</b> used to get sentence embedding link:https://www.sbert.net/examples/applications/paraphrase-mining/README.html \
3) <b>Maximum Marginal Retrival</b> to check cosine similarity across documents to consolidate candidate terms with terms across documents 

### Improvements 
#### After CV and TFIDF we can apply lemmatization with adding vectors to increase weightage across similar words with same meaning to reduce redundancy. 
#### Encoder Decoder Model with self Attention can be utilized to get better embedding with context and similarity
####  Pretrained BERT can be used to get better contextual similarity 

### [3.Visualization](#3.Visualization)
#### Motivation 
To get View of Textual Meaning and Relation of keyowrds and phrases across documents to get better intuition. 

steps
1) Used <b>KMapper</b> to visualize top keyowrds across documeents \
2) Used <b>DBScan Clustering Algorithm</b> to combine various phrases within documents AND across documents with TFIDF & Ngram

### Conclusion 
1) Here is implementation workflow for Keyphrase Extraction with visualization 
2) We can see there is duplicate and redundant terms which can be 

In [176]:
import string
import re
import numpy as np
import pandas as pd
import codecs
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from scipy import spatial


import nltk
from nltk.corpus import wordnet,stopwords
from nltk.stem import WordNetLemmatizer 
from nltk import WordNetLemmatizer, pos_tag, word_tokenize



from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

from scipy import stats

import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


import tqdm

# doc=nlp(text)

from gensim.summarization import keywords
stop_words=stopwords.words('english')
lemmatizer = WordNetLemmatizer()
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

# 1.Data_Extraction_&_Preprocessing

In [177]:
pat_data=pd.read_csv('V:/ML_projects/Merckgroup/NLP/Data/patbase_extract/patbase_bib_csv200020210716_.csv')
abs_text=pat_data['Abstract']
print(type(abs_text))

sample_data=pd.read_excel('V:/ML_projects/Merckgroup/Keyword_tagging/data/Keyword_validation.xlsx')
sample_abs_data=sample_data['Abstract']
sample_abs_data

<class 'pandas.core.series.Series'>


0      pooled crispr libraries are widely used in hig...
1      crispr loci consist of an array of short repea...
2      the simplicity and precision of crispr/cas9 sy...
3      the battle for survival between bacteria and t...
4      despite the advances made in genetic engineeri...
                             ...                        
195    crispr-cas are adaptive immune systems that de...
196    a widespread system used by bacteria for prote...
197    the cas9 nuclease from streptococcus pyogenes ...
198    the clustered regularly interspaced short pali...
199    the crispr/cas9 system was recently developed ...
Name: Abstract, Length: 200, dtype: object

In [197]:

def text_cleaning(text):
    cleaned_txt = re.sub('<[^<]+>', ' ', str(text))
    cleaned_txt = re.sub( r'\[.*?\]', ' ',cleaned_txt)  #remove brackets
#     cleaned_txt = re.sub('\d+', ' ', cleaned_txt)
#     cleaned_txt = re.sub('\(\w+\)', ' ', cleaned_txt)
#     cleaned_txt=text.translate(str.maketrans(' ', ' ',string.punctuation)) 
    return cleaned_txt
   
def stopword_removal(text):
    cleaned_txt = ' '.join([token for token in word_tokenize(text) if token not in stop_words])
    return cleaned_txt
def lemmatize_text(text):
    cleaned_txt=' '.join([lemmatizer.lemmatize(token) for token in word_tokenize(text)])

    return cleaned_txt

def preprocessed_text(df):
    
    col=['nltk_tokens','nouns_verbs','lemmatize_text']
    preprocess_txt=pd.DataFrame(columns=col)
    abs_clean_text=df.apply(text_cleaning)
    
    stopword_removed_text=abs_clean_text.apply(stopword_removal)         
    preprocess_txt['lemmatize_text']=stopword_removed_text.apply(lemmatize_text)
  
    for ind,text in preprocess_txt['lemmatize_text'].iteritems():       
        tokens=word_tokenize(text)
        pos_tokens=nltk.pos_tag(tokens)
        preprocess_txt.at[ind,'nouns_verbs']=[pos_token for pos_token,pos in pos_tokens if pos.startswith('N' or 'V')]
        preprocess_txt.at[ind,'nltk_tokens']=tokens     
    return preprocess_txt

In [198]:
# preprocessed_df=preprocessed_text(abs_text)
# preprocessed_df

preprocessed_df1=preprocessed_text(sample_abs_data)
preprocessed_df1

,nltk_tokens,nouns_verbs,lemmatize_text
0,"[pooled, crispr, library, widely, used, high-t...","[library, high-throughput, screening, study, p...",pooled crispr library widely used high-through...
1,"[crispr, locus, consist, array, short, repeat,...","[crispr, locus, array, repeat, sequence, match...",crispr locus consist array short repeat separa...
2,"[simplicity, precision, crispr/cas9, system, b...","[simplicity, precision, crispr/cas9, system, e...",simplicity precision crispr/cas9 system brough...
3,"[battle, survival, bacteria, virus, infect, (,...","[battle, survival, bacteria, phage, evolution,...",battle survival bacteria virus infect ( phage ...
4,"[despite, advance, made, genetic, engineering,...","[advance, engineering, saccharomyces, cerevisi...",despite advance made genetic engineering sacch...
...,...,...,...
195,"[crispr-cas, adaptive, immune, system, degrade...","[immune, system, element, archaea, bacteria, f...",crispr-cas adaptive immune system degrade fore...
196,"[widespread, system, used, bacteria, protectio...","[system, bacteria, protection, dna, molecule, ...",widespread system used bacteria protection pot...
197,"[cas9, nuclease, streptococcus, pyogenes, (, s...","[cas9, nuclease, streptococcus, pyogenes, spca...",cas9 nuclease streptococcus pyogenes ( spcas9 ...
198,"[clustered, regularly, interspaced, short, pal...","[repeat, gene, system, bacteria, plasmid, bact...",clustered regularly interspaced short palindro...


# 2.Keyword_&_Phrase_Extraction

### keyphrase extraction workflow 

1. Extraction of keywords using count vectorizer,Tfidf,spacy, gensim 
2. Generating Sentence embedding from Pretrained Model for each document
3. Duplicate and Redundant removal with MMR for single document ,Doc_word cosine Simililarity for Multiple document

## Single doc

In [199]:
# !pip install wget 

In [200]:
def cv_word_doc_embedding(doc):
    cv_countvec = CountVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=1).fit([doc])
    cv_words = cv_countvec.get_feature_names()
    cv_doc_embeddings = embedding_model.encode([doc])
    cv_word_embeddings =embedding_model.encode(cv_words)
    return cv_word_embeddings,cv_doc_embeddings,cv_words
    

In [201]:
def tfidf_word_doc_embedding(doc):
    tfidf_countvec = TfidfVectorizer(ngram_range=(1,2), stop_words=stop_words, min_df=0.8).fit([doc])
    tfidf_words = tfidf_countvec.get_feature_names()    

    tfidf_doc_embeddings = embedding_model.encode([doc])
    tfidf_word_embeddings = embedding_model.encode(tfidf_words)
    return tfidf_word_embeddings,tfidf_doc_embeddings,tfidf_words


In [202]:
top_n=10
diversity=0.3
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

In [203]:
def mmr(doc_embedding: np.ndarray,word_embeddings: np.ndarray,words,top_n,diversity):
    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)
    
    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
    
    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]
        
        # Update keywords & candidates
        if mmr_idx:
            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)
            
    return [(words[idx], round((word_doc_similarity.reshape(1, -1)[0][idx]), 4)) for idx in keywords_idx]

In [205]:
doc=preprocessed_df1['lemmatize_text']
cv_keywords=[]
tfidf_keywords=[]
cv_phrases=[]
tfidf_phrases=[]
for i in doc:
    cv_word_embed,cv_doc_embed,cv_words=cv_word_doc_embedding(i)
    if cv_words:
        cv_keywords_doc=mmr(cv_doc_embed,cv_word_embed,cv_words,top_n,diversity)
        cv_keywords_doc=','.join([key[0] for key in cv_keywords_doc])
        cv_phrases_doc=''.join([key[0] for key in cv_keywords_doc])
        cv_keywords.append(cv_keywords_doc)
        cv_phrases.append(cv_phrases_doc)
    
    tfidf_word_embed,tfidf_doc_embed,tfidf_words=tfidf_word_doc_embedding(i)
    tfidf_keywords_doc=mmr(tfidf_doc_embed,tfidf_word_embed,tfidf_words,5,0.2)
    tfidf_keywords_doc=','.join([key[0] for key in tfidf_keywords_doc])
    tfidf_phrases_doc=''.join([key[0] for key in tfidf_keywords_doc])
#     print('cv_keywords',cv_keywords_doc)
#     print('cv_phrases',cv_phrases_doc)
 
#     print('tfidf_keywords',tfidf_keywords_doc)
#     print('tfidf_phrases',tfidf_phrases_doc)
    
    tfidf_keywords.append(tfidf_keywords_doc)
    tfidf_phrases.append(tfidf_phrases_doc)

cv_keywords crispr,pooled,libraries,screening,throughput,cas9,gene,modified,researcher,method
cv_phrases crispr,pooled,libraries,screening,throughput,cas9,gene,modified,researcher,method
tfidf_keywords pooled crispr,crispr library,create pooled,crispr screening,gene crispr
tfidf_phrases pooled crispr,crispr library,create pooled,crispr screening,gene crispr
cv_keywords crispr,plasmid,locus,genome,deletion,virus,antisense,decoys,epidermidis,prokaryote
cv_phrases crispr,plasmid,locus,genome,deletion,virus,antisense,decoys,epidermidis,prokaryote
tfidf_keywords crispr locus,transcription crispr,crispr deletion,crispr immunity,virus plasmid
tfidf_phrases crispr locus,transcription crispr,crispr deletion,crispr immunity,virus plasmid
cv_keywords crispr,sequencing,cas9,dav
cv_phrases crispr,sequencing,cas9,dav
tfidf_keywords crispr data,crispr ng,crispr cas9,crispr dav,analyze crispr
tfidf_phrases crispr data,crispr ng,crispr cas9,crispr dav,analyze crispr
cv_keywords crispr,bacterial,phage,i

cv_keywords crispr,cpf1,bioinformatics,genome,grnas,selection,clustered,regularly,editing,rna
cv_phrases crispr,cpf1,bioinformatics,genome,grnas,selection,clustered,regularly,editing,rna
tfidf_keywords crispr cpf1,grnas crispr,efficiency crispr,cpf1 grnas,genome editing
tfidf_phrases crispr cpf1,grnas crispr,efficiency crispr,cpf1 grnas,genome editing
cv_keywords crispr,biotechnologies,genome,improved,cas,targeting,tool,nanotechnology,editing,robust
cv_phrases crispr,biotechnologies,genome,improved,cas,targeting,tool,nanotechnology,editing,robust
tfidf_keywords crispr technology,advancement crispr,development crispr,crispr cas,application crispr
tfidf_phrases crispr technology,advancement crispr,development crispr,crispr cas,application crispr
cv_keywords crispr,rnai,gene,cas9,therapy,modulation,biomedical,genome,protein,combination
cv_phrases crispr,rnai,gene,cas9,therapy,modulation,biomedical,genome,protein,combination
tfidf_keywords rnai crispr,gene therapy,cas9 crispr,capability cr

cv_keywords genome,crispr,profiling,rna,genomic,transcriptomic,epigenomic,protein,facile,cas
cv_phrases genome,crispr,profiling,rna,genomic,transcriptomic,epigenomic,protein,facile,cas
tfidf_keywords programmability crispr,genomic transcriptomic,rna guided,crispr clustered,genome programmability
tfidf_phrases programmability crispr,genomic transcriptomic,rna guided,crispr clustered,genome programmability
cv_keywords crispr,labeling,chromatin,genome,cas9,microscopy,binding,targeting,catalytically,optogenetic
cv_phrases crispr,labeling,chromatin,genome,cas9,microscopy,binding,targeting,catalytically,optogenetic
tfidf_keywords crispr labeling,crispr cas9,binding crispr,inducible crispr,chromatin architecture
tfidf_phrases crispr labeling,crispr cas9,binding crispr,inducible crispr,chromatin architecture
cv_keywords crispr3,enterococcus,multidrug,genome,resistance,plasmid,clustered,codistribution,isolates,mdr
cv_phrases crispr3,enterococcus,multidrug,genome,resistance,plasmid,clustered,cod

cv_keywords crispr,acrif9,dsdna,protein,genetic,nucleic,inhibit,phage,acr,bind
cv_phrases crispr,acrif9,dsdna,protein,genetic,nucleic,inhibit,phage,acr,bind
tfidf_keywords crispr acrif1,crispr rna,inhibit crispr,crispr protein,block dna
tfidf_phrases crispr acrif1,crispr rna,inhibit crispr,crispr protein,block dna
cv_keywords crispr,prokaryote,virus,crrnas,genome,archaeal,bacteria,controlled,immunity,nuclease
cv_phrases crispr,prokaryote,virus,crrnas,genome,archaeal,bacteria,controlled,immunity,nuclease
tfidf_keywords crispr archaea,crispr rna,genomic crispr,prokaryote constantly,attack virus
tfidf_phrases crispr archaea,crispr rna,genomic crispr,prokaryote constantly,attack virus
cv_keywords crispr,future,genetic,chemist,frontier,research,rapid,creative,pressing,genome
cv_phrases crispr,future,genetic,chemist,frontier,research,rapid,creative,pressing,genome
tfidf_keywords crispr researchers,engineering crispr,approach genetic,future genome,crispr
tfidf_phrases crispr researchers,engin

cv_keywords crispr,cas12a,genome,cpf1,immune,bioinformatic,microbial,targeting,nuclease,protein
cv_phrases crispr,cas12a,genome,cpf1,immune,bioinformatic,microbial,targeting,nuclease,protein
tfidf_keywords cpf1 crispr,crispr cas12a,crisprs microbial,genome editing,cas9 crispr
tfidf_phrases cpf1 crispr,crispr cas12a,crisprs microbial,genome editing,cas9 crispr
cv_keywords crispr,rna,seminal,bacteria,gene,prokaryote,system,cas,dna,type
cv_phrases crispr,rna,seminal,bacteria,gene,prokaryote,system,cas,dna,type
tfidf_keywords crispr system,rna targets,tool bacteria,seminal study,gene editing
tfidf_phrases crispr system,rna targets,tool bacteria,seminal study,gene editing
cv_keywords crispr,sequencing,perturb,bacterial,component,ca,gene,cas,editing,genome
cv_phrases crispr,sequencing,perturb,bacterial,component,ca,gene,cas,editing,genome
tfidf_keywords crispr ca,crispr genetic,adaptation crispr,crispr knockout,referred crispr
tfidf_phrases crispr ca,crispr genetic,adaptation crispr,crispr k

cv_keywords bacteria,crispr,archaea,rna,cas9,immune,nucleic,metagenomic,virus,plasmid
cv_phrases bacteria,crispr,archaea,rna,cas9,immune,nucleic,metagenomic,virus,plasmid
tfidf_keywords archaea crispr,bacteria archaea,rna guided,bacteriophage virus,cas9 protein
tfidf_phrases archaea crispr,bacteria archaea,rna guided,bacteriophage virus,cas9 protein
cv_keywords crispr,nuclease,genome,chromatin,modulating,tool,dna,versatile,clustered,repeat
cv_phrases crispr,nuclease,genome,chromatin,modulating,tool,dna,versatile,clustered,repeat
tfidf_keywords application crispr,elevate crispr,engineering crispr,crispr including,crispr tool
tfidf_phrases application crispr,elevate crispr,engineering crispr,crispr including,crispr tool
cv_keywords crispr,archaea,immunity,genetic,adaptive,phage,inactivate,bacteria,proteins,widespread
cv_phrases crispr,archaea,immunity,genetic,adaptive,phage,inactivate,bacteria,proteins,widespread
tfidf_keywords crispr proteins,adaptive immune,bacteria archaea,inactivate 

cv_keywords crispr,biotechnology,viral,gene,optimized,regularly,cas,biochem,editing,genome
cv_phrases crispr,biotechnology,viral,gene,optimized,regularly,cas,biochem,editing,genome
tfidf_keywords crispr biological,development crispr,crispr systems,crispr versatile,genome editing
tfidf_phrases crispr biological,development crispr,crispr systems,crispr versatile,genome editing
cv_keywords crispr,genome,treatment,ophthalmology,screening,modification,vivo,mutagenesis,cas,zebrafish
cv_phrases crispr,genome,treatment,ophthalmology,screening,modification,vivo,mutagenesis,cas,zebrafish
tfidf_keywords improvement crispr,genome surgery,crispr associated,genome editing,repeat crispr
tfidf_phrases improvement crispr,genome surgery,crispr associated,genome editing,repeat crispr
cv_keywords crispr,nuclease,dcas9,gene,repurposed,cas9,clustered,epigenetic,genome,rna
cv_phrases crispr,nuclease,dcas9,gene,repurposed,cas9,clustered,epigenetic,genome,rna
tfidf_keywords dcas9 crispr,cas9 crispr,editing nuc

cv_keywords cloning,sgrna,crispr,gene,plasmid,cas9,modify,sccrispr,recombination,genomic
cv_phrases cloning,sgrna,crispr,gene,plasmid,cas9,modify,sccrispr,recombination,genomic
tfidf_keywords gene editing,sgrna plasmid,cloning crispr,crispr gene,plasmid cloning
tfidf_phrases gene editing,sgrna plasmid,cloning crispr,crispr gene,plasmid cloning
cv_keywords crispr,prevotella,genome,cpf1,repeat,clustered,editing,cas9,protein,francisella
cv_phrases crispr,prevotella,genome,cpf1,repeat,clustered,editing,cas9,protein,francisella
tfidf_keywords crispr prevotella,efficient genome,repeat crispr,protein crispr,crispr cas9
tfidf_phrases crispr prevotella,efficient genome,repeat crispr,protein crispr,crispr cas9
cv_keywords crispr,mirnas,inhibited,cas9,gene,rnai,editing,enoxacin,enhancer,targeting
cv_phrases crispr,mirnas,inhibited,cas9,gene,rnai,editing,enoxacin,enhancer,targeting
tfidf_keywords inhibited crispr,rnai enhancer,artificial mirnas,crispr cas9,genome editing
tfidf_phrases inhibited cr

cv_keywords crispr,archaeal,immune,nucleic,intergenomic,crrnas,targeting,likely,rna,cmr
cv_phrases crispr,archaeal,immune,nucleic,intergenomic,crrnas,targeting,likely,rna,cmr
tfidf_keywords archaeal crispr,crispr rna,crrnas targeting,based immune,crispr locus
tfidf_phrases archaeal crispr,crispr rna,crrnas targeting,based immune,crispr locus
cv_keywords crispr,cas6,rna,crea,toxin,bacteriostatic,immunity,regulates,gene,protein
cv_phrases crispr,cas6,rna,crea,toxin,bacteriostatic,immunity,regulates,gene,protein
tfidf_keywords crispr rna,crea crispr,bacterial crispr,cas regulation,crispr cas
tfidf_phrases crispr rna,crea crispr,bacterial crispr,cas regulation,crispr cas
cv_keywords genome,recombination,crispr,deletion,chromosomal,pcd,replacement,strand,manipulating,homologous
cv_phrases genome,recombination,crispr,deletion,chromosomal,pcd,replacement,strand,manipulating,homologous
tfidf_keywords genome manipulation,chromosomal deletion,versatile genome,homologous recombination,genome edit

## Multiple  documents

In [206]:
import numpy as np
import itertools
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Tuple
import nltk
from nltk.corpus import stopwords
# from nltk.tokenize import sentence
from sklearn.feature_extraction.text import TfidfVectorizer


from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")



## Count Vectorizer

In [191]:
def phrase_generator(docs,words,countvec):
    
    top_n:int=10
    diversity:0.5
    df = countvec.transform(docs)
    
    keywords=[]    
    phrases=[]
    
    
    for index, doc in enumerate(docs):
        doc_words = [words[i] for i in df[index].nonzero()[1]]
        if doc_words:
            doc_word_embeddings = np.array([cv_word_embeddings[i] for i in df[index].nonzero()[1]])
            distances = cosine_similarity([cv_doc_embeddings[index]], doc_word_embeddings)[0]
            doc_keywords = [(doc_words[i], round(float(distances[i]), 4)) for i in distances.argsort()[-top_n:]]
            doc_keywords=','.join([key[0] for key in doc_keywords])
            doc_phrases=''.join([key[0] for key in doc_keywords])
            keywords.append(doc_keywords)
            phrases.append(doc_phrases)                                 
    return keywords,phrases
                              

In [ ]:
# docs=preprocessed_df['lemmatize_text']
docs1=preprocessed_df1['lemmatize_text']
# Nouns_docs=preprocessed_df1['nouns_verbs'].apply(lambda x:' '.join(x))

CV_countvec = CountVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=1).fit(docs1)
CV_words = CV_countvec.get_feature_names()

cv_doc_embeddings = embedding_model.encode(docs1)
cv_word_embeddings =embedding_model.encode(CV_words)

# cv_keywords,cv_phrases=countvec_phrase_generator(docs)
cv_keywords_doc,cv_phrases_doc=phrase_generator(docs1,CV_words,CV_countvec)



## TFIDF


In [ ]:

tfidf_countvec = TfidfVectorizer(ngram_range=(1,1), stop_words=stop_words, min_df=1).fit(docs1)
tfidf_words = tfidf_countvec.get_feature_names()

tfidf_doc_embeddings = embedding_model.encode(docs1)
tfidf_word_embeddings = embedding_model.encode(tfidf_words)

tfidf_keywords_doc,tfidf_phrases_doc=phrase_generator(docs1,tfidf_words,tfidf_countvec)



In [ ]:
sample_data['mmr_cv_keywords']=cv_keywords
sample_data['mmr_tfidf_bigram_keywords']=tfidf_keywords
sample_data['cv_doc_keywords']=cv_phrases_doc
sample_data['tfidf_doc_keywords']=tfidf_phrases_doc

In [ ]:
sample_data.to_excel('V:/ML_projects/Merckgroup/Keyword_tagging/data/Keyword_validation_output_final.xlsx')
sample_data

# 3.Visualization

## KEPPLER MAPPER
### A scikit library that provides the functionality to project data visually, to cluster it, and to create connections between it.
#### source: git clone https://github.com/MLWave/kepler-mapper


### Goal : 
  1) Convert Documents into Features matrix \
  2) Phrase combine with keywords for Projection \
  3) Check keywords or phrases which are common and uncommon across all documents \
  
### Steps:

 1) List of 2D representations of each of our sentences into features using TFIDF
 2) Dimensionality Reduction: TruncatedSVD , Isomap \
 3) Cluster: DBSCAN we’re going to cluster the Phrase sentences \
 4) Common Keyphrase in Cluster : TF-IDF ngrams (1-3) to help us identify each cluster i.e.Phrases of length 3 which are most common.
 
### Benefits
1. The sentences in a cluster do have overlap in meaning (that’s why they’re clustered together)
2. To get overview of various and most important phrases with feature words that are group together 
 
 ### Analysis of Graph and Usage
 1. Node Label: Most common word or Phrase in all sentence in that cluster
 2. + Cluster details: Above and below average feature words with Members associated and with their    
 3. Links shows relation across keywords across the clusters 
 
## Takeaway Points and Further steps
### 1. As we can still see same word has associated with different cluster. 
### 2. Development with Analysis of this graph will be crucial step to bring contextual changes with our workflow in timely manner. 
### 3. Variation in input documents will be analyzed i.e. document with Noun Phrase joined to be feeded in Graph.
### 4. Sentence similarity will be Replaced <b>Universal sentence encoder</b> while taking to Development phase
### 5. BERT Pretrained model can be used for preparing vectors with development phase.
### 6. Generalized Encoder and Decoder can be Productionized which can be useful in other modules 

In [ ]:
import kmapper as km
# Project all of the "sentences" down into a 2D representation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import Isomap
from sklearn.preprocessing import MinMaxScaler
from sklearn import cluster

In [ ]:
def keyphrase_mapper(phrases):
    mapper = km.KeplerMapper(verbose=2)

    projected_X = mapper.fit_transform(np.array(phrases),
    projection=[TfidfVectorizer(analyzer="char",
                            ngram_range=(1,6),
                            max_df=0.85,
                            min_df=0.05),
            TruncatedSVD(n_components=100,
                         random_state=1729),
            Isomap(n_components=2,
                   n_jobs=-1)],
    scaler=[None, None, MinMaxScaler()])

    print("SHAPE",projected_X.shape)

    return projected_X


In [192]:
# Cluster the projected data
def cluster_visualization(docs,projected_X,path):
    mapper = km.KeplerMapper(verbose=2)
    graph = mapper.map(projected_X, clusterer=cluster.DBSCAN(eps=0.5, min_samples=3))
    # Get counts of phrases from data (to be used for the label and for cluster statistics)
    vec = TfidfVectorizer(analyzer="word",
                  strip_accents="unicode",
                  stop_words="english",
                  ngram_range=(1,3),
                  max_df=0.9,
                  min_df=0.02)

    interpretable_inverse_X = vec.fit_transform(docs).toarray()
    interpretable_inverse_X_names = vec.get_feature_names()

    print("SHAPE", interpretable_inverse_X.shape)
    print("FEATURE NAMES SAMPLE",
    interpretable_inverse_X_names[:400])
    html = mapper.visualize(graph,X=interpretable_inverse_X,
                        X_names=interpretable_inverse_X_names,
                        path_html=path,
                        lens=projected_X,
                        lens_names=["ISOMAP1", "ISOMAP2"],
                        title="Visualizing Text - Abstract",
                        )


In [193]:
projected_X=keyphrase_mapper(cv_phrases)
projected_tfidf_X=keyphrase_mapper(tfidf_phrases)

cluster_visualization(docs1,projected_X,'Abstract_CV_mmr.html')
cluster_visualization(docs1,projected_tfidf_X,'Abstract_tfidf_mmr.html')

KeplerMapper(verbose=2)
..Composing projection pipeline of length 3:
	Projections: TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.85, max_features=None,
                min_df=0.05, ngram_range=(1, 6), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
		TruncatedSVD(algorithm='randomized', n_components=100, n_iter=5,
             random_state=1729, tol=0.0)
		Isomap(eigen_solver='auto', max_iter=None, metric='minkowski',
       metric_params=None, n_components=2, n_jobs=-1, n_neighbors=5,
       neighbors_algorithm='auto', p=2, path_method='auto', tol=0)
	Distance matrices: False
False
False
	Scalers: None
None
MinMaxScaler(copy=True, feature_range=(0, 1))
..Pro

Wrote visualization to: Abstract_CV_mmr.html
KeplerMapper(verbose=2)
Mapping on data shaped (200, 2) using lens shaped (200, 2)

Minimal points in hypercube before clustering: 3
Creating 100 hypercubes.
   > Found 1 clusters in hypercube 0.
Cube_1 is empty.

Cube_2 is empty.

Cube_3 is empty.

Cube_4 is empty.

Cube_5 is empty.

Cube_6 is empty.

   > Found 1 clusters in hypercube 7.
   > Found 1 clusters in hypercube 8.
Cube_9 is empty.

   > Found 1 clusters in hypercube 10.
Cube_11 is empty.

   > Found 1 clusters in hypercube 12.
   > Found 1 clusters in hypercube 13.
Cube_14 is empty.

   > Found 1 clusters in hypercube 15.
Cube_16 is empty.

Cube_17 is empty.

   > Found 1 clusters in hypercube 18.
   > Found 1 clusters in hypercube 19.
   > Found 1 clusters in hypercube 20.
   > Found 1 clusters in hypercube 21.
   > Found 1 clusters in hypercube 22.
   > Found 1 clusters in hypercube 23.
Cube_24 is empty.

Cube_25 is empty.

Cube_26 is empty.

   > Found 1 clusters in hypercube

Wrote visualization to: Abstract_tfidf_mmr.html


In [207]:
projected_X=keyphrase_mapper(cv_phrases_doc)
projected_tfidf_X=keyphrase_mapper(tfidf_phrases_doc)

cluster_visualization(docs1,projected_X,'Abstract_CV_doc.html')
cluster_visualization(docs1,projected_tfidf_X,'Abstract_tfidf_doc.html')


KeplerMapper(verbose=2)
..Composing projection pipeline of length 3:
	Projections: TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.85, max_features=None,
                min_df=0.05, ngram_range=(1, 6), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)
		TruncatedSVD(algorithm='randomized', n_components=100, n_iter=5,
             random_state=1729, tol=0.0)
		Isomap(eigen_solver='auto', max_iter=None, metric='minkowski',
       metric_params=None, n_components=2, n_jobs=-1, n_neighbors=5,
       neighbors_algorithm='auto', p=2, path_method='auto', tol=0)
	Distance matrices: False
False
False
	Scalers: None
None
MinMaxScaler(copy=True, feature_range=(0, 1))
..Pro

ValueError: could not convert string to float: 'sgrnas,crispr,genome,efficient,cas9,design,regulation,editing,targeted,bioinformatics'

In [195]:
from kmapper import jupyter
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 64em; }</style>"))

jupyter.display(path_html="Abstract_CV_mmr.html")


C:\Users\HP\anaconda3\lib\site-packages\IPython\core\display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [196]:
from kmapper import jupyter
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

jupyter.display(path_html="Abstract_tfidf_mmr.html")